## TJ's Item Nutrition Visualization

#### Load HP Store Data

In [1]:
import polars as pl
tjhp_clean = pl.read_csv("data/tjhp_clean.csv")

### Hyde Park Viz

In [ ]:
import altair as alt

# alt.Chart(tjhp_clean).mark_point().encode(
#     x='price_usd',
#     y='calories_per_container',
#     color='calories_per_serving',
#     tooltip=['sku', 'name']
# ).interactive()

# https://github.com/vega/altair/issues/1245
# How to disable automatic alphabetic sorting of x value
alt.Chart(tjhp_clean.sort(by='dollars_per_calorie', descending=False).head(20)).mark_bar().encode(
    # x='name',
    x=alt.X('name', title='Product', type='nominal', sort=None, axis=alt.Axis(labelAngle=-45)),
    y=alt.Y(field='dollars_per_100calorie', type='quantitative', title='$ per 100 Calories'),
    color=alt.Color('dollars_per_serving', title='$ per Serving').scale(scheme="greens"),
    tooltip=['sku', 'name', 'price_usd', 'dollars_per_calorie', 'servings_per_container']
).transform_calculate(
    dollars_per_100calorie='datum.dollars_per_calorie * 100' # Calculate the new field
).interactive(

).properties(
    title='Hyde Park TJ\'s 20 items with the cheapest calories',
    width=800,
    height=300
).configure_title(
    fontSize=24,
    # font='IMPACT',
    anchor='middle',
    color='black'
).configure_axis(
    labelFontSize=15,
    titleFontSize=20
)

# tjhp_clean.columns

alt.Chart(...)

In [ ]:
pretzel = [i for i in all_items_hp if i['sku']=='079176']

In [ ]:
len(pretzel)

1

In [ ]:
tjhp['nutrition'][0][0]['details']

[{'amount': '38 g',
  'display_seq': 1,
  'nutritional_item': 'Total Fat',
  'percent_dv': '.49'},
 {'amount': '25 g',
  'display_seq': 2,
  'nutritional_item': 'Saturated Fat',
  'percent_dv': '1.25'},
 {'amount': '0 g',
  'display_seq': 3,
  'nutritional_item': 'Trans Fat',
  'percent_dv': ''},
 {'amount': '0 mg',
  'display_seq': 4,
  'nutritional_item': 'Cholesterol',
  'percent_dv': '0'},
 {'amount': '500 mg',
  'display_seq': 5,
  'nutritional_item': 'Sodium',
  'percent_dv': '.22'},
 {'amount': '58 g',
  'display_seq': 6,
  'nutritional_item': 'Total Carbohydrate',
  'percent_dv': '.21'},
 {'amount': '4 g',
  'display_seq': 7,
  'nutritional_item': 'Dietary Fiber',
  'percent_dv': '.14'},
 {'amount': '4 g',
  'display_seq': 8,
  'nutritional_item': 'Total Sugars',
  'percent_dv': ''},
 {'amount': '0 g Added Sugars',
  'display_seq': 9,
  'nutritional_item': 'Includes',
  'percent_dv': '0'},
 {'amount': '11 g',
  'display_seq': 10,
  'nutritional_item': 'Protein',
  'percent_dv':

### All list

In [ ]:
import polars as pl

In [ ]:
# pl.json_normalize(all_items_fixed[100:102])

Beyond just dumping each json object from `all_items_fixed` into `pl.json_normalize()`, we also need to further handle the `nutrition` and `ingredients` dictionaries.

In [ ]:
pl.json_normalize(all_items_fixed[100]['nutrition'])

For now, just save it as expressly as possible.

In [ ]:
# df_all_items_raw = pl.json_normalize(all_items_fixed,
#                                      infer_schema_length=None)

In [ ]:
df_all_items_raw.tail(3)
# Columns 2 and 3 are Lists of Structs of...


In [ ]:
# df_all_items_raw.write_csv('data/all_items_raw.csv')
# # Nested columns 2 and 3 do not work with CSV

In [ ]:
# import json
# all_items_fixed_raw_path = "data/all_items_fixed_raw.json"
# with open(all_items_fixed_raw_path, "w") as f:
#        json.dump(all_items_fixed, f, indent=4)



Every description is empty html???

In [ ]:
broc = [item['name'] for item in all_items_fixed if 'KALE' in item['name']]



broc

In [ ]:
df_all_items_raw.write_csv()

Let's try and isolate only food items.

In [ ]:
pl.json_normalize(all_items_fixed[100], max_level=4)

In [ ]:
nuts = [item['nutrition'] for item in all_items_fixed]

In [ ]:
# Get only items with non-empty nutrition information
# This will be criterion for what counts as "food"

In [ ]:
# nut_lens = [len(nut) for nut in nuts]

Oh, sku 10032021 "POP UP SPONGES" for some reason contains information for pizza crusts? The problematic `"item_description": "Broccoli & Kale Pizza Crust description"`.

#### Getting only the values we need from the json, making it relational

In [ ]:
import json
import polars as pl

In [ ]:
with open('data/all_items_fixed_raw.json', 'r') as f:
    tj_raw = json.load(f)

Recall that the `item_description` field is `null` for just about every entry except one where it is instead the unrelated `Broccoli & Kale Pizza Crust description`.

In [ ]:
tj_raw = [item for item in tj_raw if item['item_description'] != 'Broccoli & Kale Pizza Crust description']

In [ ]:
tj = pl.DataFrame(tj_raw)

Need to:

* Remove rows with null `nutrition`
* Remove useless fields (`item_description`, `description`)
* Turn nested fields into 
    * lists (ingredients)
    * individual and possibly sparse columns (fiber, protein, calories)

In [ ]:
tj = tj.drop(['item_description', 'description'])

In [ ]:
tj.shape

In [ ]:
tj = tj.drop_nulls(subset=['nutrition', 'price'])
tj.shape

In [ ]:
tj.head()

In [ ]:
tj['price'].str

In [ ]:
temp = tj.with_columns(
    pl.col("price").struct.field("regularPrice").struct.field("amount").struct.field("currency").alias("nested_field")
)

print(temp)

In [ ]:
temp['nested_field'].value_counts()

OK, they are all in USD, proceed.

In [ ]:
tj = tj.with_columns(
    pl.col("price").struct.field("regularPrice").struct.field("amount").struct.field("value").alias("price_usd")
)


Onto the same nested json but for calories...

Wow, these nutrition items are formatted horrendously. The key is not a key whatsoever. Should go back and fix this in the original GraphQL query under `nutrition`.

In [ ]:
tj['nutrition'][0]

Oh, maybe not.

In [ ]:
tj['nutrition'][0][0]

In [ ]:
tj['nutrition'][0][0]['calories_per_serving']

In [ ]:
tj = tj.with_columns(
    # tj['nutrition']
    pl.col("nutrition").map_elements(lambda x: x[0]['calories_per_serving'] if len(x)>0 else None, return_dtype=str).alias("calories_per_serving")
)
tj.head(3)

These things definitely do not actually have 0 calories. Drop them.

In [ ]:
tj.filter((pl.col("calories_per_serving") == ""))

In [ ]:
tj = tj.filter((pl.col("calories_per_serving") != ""))

More problematic calories per serving fields: `8 out of 4570 values: ["3291 kcal/kg; 29 kcal/treat", "3200 kcal/kg; 18 kcal/treat", … "varied"]`.

In [ ]:
tj.shape

In [ ]:
### DROP THEM. come back later to salvage whichever.
tj = tj.filter(tj['calories_per_serving'].str.contains('^\\s*\\d+\\s*$'))
tj.shape
# Yes, this filters out those 8 values giving us trouble before

In [ ]:
tj = tj.with_columns(
    pl.col('calories_per_serving').str.strip_chars(' ').cast(pl.Float32).alias('calories_per_serving')
)

Now same for servings per container....

In [ ]:
tj['nutrition'][0][0]['servings_per_container']

In [ ]:
# Create raw servings column
tj = tj.with_columns(
    pl.col("nutrition").map_elements(lambda x: x[0]['servings_per_container'] if len(x)>0 else None, return_dtype=str).alias("servings_per_container")
)

In [ ]:
unique_spc = tj['servings_per_container'].unique() # alue_counts()['servings_per_container']
for spc in unique_spc:
    print(spc)

In [ ]:
tj.shape

In [ ]:
# tj.head(3)

The common problems with servings per container include:
* `Serves one` and `Serves does not need this since there is only one serving` (manually replace with 1)
* `Serves about 3 (About 2.5 without Dressing)` (just grab first numeric)
* `Serves 8 on 8oz and 12 on 12oz` (This might be a problem depending on whether there is a single reported calorie value)
* `Serves varied` (idek man)

In [ ]:

tj = tj.with_columns(
    spc=pl.when(pl.col("servings_per_container").str.contains(' one', literal=True))
    .then(1)
    .otherwise(pl.col("servings_per_container"))
)#.filter(tj['servings_per_container'].str.contains(' one', literal=True))

In [ ]:
# (?i)(serves about)|(serves)|(about)
tj = tj.with_columns(
    spc=tj['spc'].str.extract(r'(\d+)').cast(pl.Float32)
)

In [ ]:
tj = tj.with_columns(
    calories_per_container = tj['calories_per_serving'] * tj['spc']
)

In [ ]:
tj = tj.with_columns(
    dollars_per_calorie = tj['price_usd'] / tj['calories_per_container']
)

In [ ]:
tj = tj.with_columns(
    calories_per_dollar = 1 / pl.col('dollars_per_calorie')
)

In [ ]:
tj.columns

In [ ]:
# Remove 0-calorie nothings like salt and tea and hot sauce
# Remove items that are just fucking free i guess??!!
tj_trim = tj[:, ['sku', 'name', 'calories_per_dollar', 'dollars_per_calorie', 'price_usd', 'calories_per_container', 'calories_per_serving', 'spc', ]].filter(
    ~tj['dollars_per_calorie'].is_null() & ~tj['dollars_per_calorie'].is_nan() & (tj['calories_per_container']!=0) & (tj['price_usd']!=0)
)
tj_trim

Lol why are there polars `null` and also just `NaN` brother

In [ ]:
tj_trim.sort(by='dollars_per_calorie', descending=False)
# Chocolatey dipping kit limited time only, caloric maximum is a fucking cryptid, it's lost media
# NO these fucking prices are just straight up wrong, 1 cent???

In [ ]:
cfg = pl.Config()
cfg.set_tbl_rows(20)
# with pl.Config(tbl_rows=10):
#     tj_trim.sort(by='dollars_per_calorie', descending=False)
tj_trim.sort(by='calories_per_dollar', descending=True).head(20)
# How can I do a HAVING filter after the sort? To remove crazy low dollars_per_calorie values
## What did I mean by this. Maybe HAVING after the calories_per_container thing?

In [ ]:
# Energy Bar Peanut Butter / TJ'S CRUNCHY PEANUT BUTTER ENERGY
# Wrong, it is listing price of a single bar, but servings as 12 bars!!!!
tj.filter(tj['sku'] == '077616')

# Same issue with TJ'S CHOCOLATE CHIP ENERGY

In [ ]:
tj.filter(tj['sku'] == '070379')

In [ ]:
tj.filter(tj['sku'] == '073468')

Let's filter out products that obviously make no sense, e.g. excessive calories in a container and $0.01 price tags.

In [ ]:
tj_clean = tj_trim.clone() # Deep copy

In [ ]:
tj_clean = tj_clean.sort(by='calories_per_dollar', descending=True)


In [ ]:
tj_clean['price_usd'].hist()

In [ ]:
tj_clean.filter(tj_clean['price_usd'] <= 0.49)

In [ ]:
# tj = tj.with_columns(
#     calories_per_dollar = 1 / pl.col('dollars_per_calorie')
# )

# df.filter REMOVES (filters lol) things that satisfy the condition
tj_clean = tj_clean.filter(
        (tj_clean['calories_per_container'] < 30000) # Demon core candied ginger
        &
        (tj_clean['price_usd'] >= 0.49) # Threshold at fruit wraps and "just a handful"-type products
)

In [ ]:
# tj.filter(~tj['servings_per_container'].str.contains('[(serves)(Serves)(about)]'))

In [ ]:
# tj_clean_top20 = 
# tj_clean.sort(by='calories_per_dollar', descending=True).head(20)

To sanity check items that do not have accurate servings per container given the price, we can insert another variable for the price per serving to see if that is weird.

In [ ]:
tj_clean = tj_clean.with_columns(
    dollars_per_serving = tj_clean['price_usd'] / tj_clean['spc']
)

In [ ]:
tj_clean.head(3)

Also, let's fix some obvious mistakes manually.

In [ ]:
tj_clean = tj_clean.with_columns(
    pl.when(tj_clean['sku'].is_in(["077616", "077618"]))
    .then(1)
    .otherwise(pl.col("spc"))
    .alias("spc")
)


In [ ]:
tj_clean = tj_clean.with_columns(
    calories_per_container = tj_clean['calories_per_serving'] * tj_clean['spc']
)

In [ ]:
tj_clean = tj_clean.with_columns(
    calories_per_dollar = tj_clean['calories_per_container'] / tj_clean['price_usd']
)

In [ ]:
tj_clean = tj_clean.with_columns(
    dollars_per_calorie = 1 / tj_clean['calories_per_dollar']
)

In [ ]:
tj_clean = tj_clean.sort(by='calories_per_dollar', descending=True)

Save.

In [ ]:
tj_clean.write_csv("data/tj_clean.csv")

Load.

In [ ]:
tj_clean

## Viz

In [ ]:
import altair as alt

In [ ]:

alt.Chart(tj_clean).mark_point().encode(
    x='price_usd',
    y='calories_per_container',
    color='calories_per_serving',
    tooltip=['sku', 'name']
).interactive()

In [ ]:
# https://github.com/vega/altair/issues/1245
# How to disable automatic alphabetic sorting of x value
alt.Chart(tj_clean.sort(by='calories_per_dollar', descending=True).head(20)).mark_bar().encode(
    # x='name',
    x=alt.X('name', type='nominal', sort=None, axis=alt.Axis(labelAngle=-45)),
    y='calories_per_dollar',
    color=alt.Color('dollars_per_serving').scale(scheme="greens"),
    tooltip=['sku', 'name', 'price_usd', 'calories_per_serving', 'spc']
).interactive(

).properties(
    width=800,
    height=300
)

In [ ]:
tj_clean.columns

## Misc

In [ ]:
import pandas as pd

In [ ]:
ptj = pd.DataFrame(tj)

In [ ]:
ptj.columns = tj.columns

In [ ]:
ptj.head(3)

In [ ]:
ptj['nutrition'][0][0]

In [ ]:
ptj

In [ ]:
# ptj['nutrition'].apply(lambda x: x[0] if len(x)>0 else None)

In [ ]:
# tj['nutrition'].struct.field('calories_per_serving')

In [ ]:
# tj.with_columns(
#     pl.col("nutrition").map_elements(lambda x: x[0][0]).struct.field('calories_per_serving').alias("calories_per_serving")
# )

In [ ]:
# temp = tj.with_columns(
#     pl.col("nutrition").struct.field("calories_per_serving").alias("calories_per_serving")
# )
# temp